<a href="https://colab.research.google.com/github/kluless13/paper/blob/main/yolov7_COTs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install dependencies

In [ ]:
!nvidia-smi

Sun Aug 27 14:08:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
HOME = "/content"
print(HOME)

/content


## Requirements [reqs1.txt]

## Base
* matplotlib>=3.2.2
* numpy>=1.18.5
* opencv-python>=4.1.1
* Pillow>=7.1.2
* PyYAML>=5.3.1
* requests>=2.23.0
* scipy>=1.4.1
* torch>=1.7.0,!=1.12.0
* torchvision>=0.8.1,!=0.13.0
* tqdm>=4.41.0
* protobuf<4.21.3

## Logging
* tensorboard>=2.4.1
* wandb

## Plotting
* pandas>=1.1.4
* seaborn>=0.11.0

## Export
* coremltools>=4.1 [CoreML export]
* onnx>=1.9.0 [ONNX export]
* onnx-simplifier>=0.3.6 [ONNX simplifier]
* scikit-learn==0.19.2 [CoreML quantization]
* tensorflow>=2.4.1 [TFLite export]
* tensorflowjs>=3.9.0 [TF.js export]
* openvino-dev [OpenVINO export]

## Extras
* ipython [interactive notebook]
* psutil [system utilization]
* thop [FLOPs computation]
* albumentations>=1.0.3
* pycocotools>=2.0 [COCO mAP]
* roboflow

In [ ]:
!pip install -r reqs1.txt
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.6 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=f31db4fec27dd4d53b2693caaefcf6ffe6af8c5c817b66cab4a79148884db123
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      

## 2. Path to training  
5 versions of YOLOv7 = 5 directories with the train/val/test.py scripts

### 2.1. Training on the dataset

In [ ]:
# Setting path to operational directory

# For yolov7 base:

%cd {HOME}
%mkdir yolov7-base
%cd {HOME}/yolov7-base
!gitclone https://github.com/WongKinYiu/yolov7.git

/content
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to lionfish-1 in yolov7pytorch:: 100%|██████████| 1922/1922 [00:00<00:00, 4972.91it/s]


In [ ]:
# Getting the weights too
!wget -P {HOME}/yolov7-base https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

# Roboflow API into yolov7-base

from roboflow import Roboflow
rf = Roboflow(api_key="2dnFoL2vjwfFBNkIUxLZ")
project = rf.workspace("hunter-gunter").project("lionfish-sserd")
dataset = project.version(1).download("yolov7")

In [ ]:
# Now that we have our data and scripts in the same place, we run training.

!python train.py --batch 16 --cfg cfg/training/yolov7.yaml --epochs 55 --data {dataset.location}/data.yaml --weights 'yolov7.pt' --device 0

### 2.2. Running evaluation metrics

The evaluation metrics are F1, mAP, P, R, PR and Accuracy.

In [ ]:
%cd {HOME}/yolov7-base

from IPython.display import Image

display(Image("{HOME}/yolov7/runs/train/exp/F1_curve.png", width=400, height=400)) #F1 score
display(Image("{HOME}/yolov7/runs/train/exp/PR_curve.png", width=400, height=400)) #PR curve
display(Image("{HOME}/yolov7/runs/train/exp/confusion_matrix.png", width=500, height=500)) # Confusion matrix
display(Image("{HOME}/yolov7/runs/train/exp/Accuracy.png", width=500, height=500)) # Accuracy
display(Image("{HOME}/yolov7/runs/train/exp/P_curve.png", width=500, height=500)) # Precision
display(Image("{HOME}/yolov7/runs/train/exp/R_curve.png", width=500, height=500)) # Recall
display(Image("{HOME}/yolov7/runs/train/exp/mAP_curve.png", width=500, height=500)) # mAP score

### 2.3. Running inferences on custom video

Import video footage to be able to witness the code in action.

In [ ]:
%cd {HOME}/yolo7-base

video_path = ""

In [ ]:
# Initializing video object
video = cv2.VideoCapture(video_path)


#Video information
fps = video.get(cv2.CAP_PROP_FPS)
w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
nframes = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialzing object for writing video output
output = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'DIVX'),fps , (w,h))
torch.cuda.empty_cache()
# Initializing model and setting it for inference
with torch.no_grad():
  weights, imgsz = opt['weights'], opt['img-size']
  set_logging()
  device = select_device(opt['device'])
  half = device.type != 'cpu'
  model = attempt_load(weights, map_location=device)  # load FP32 model
  stride = int(model.stride.max())  # model stride
  imgsz = check_img_size(imgsz, s=stride)  # check img_size
  if half:
    model.half()

  names = model.module.names if hasattr(model, 'module') else model.names
  colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
  if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

  classes = None
  if opt['classes']:
    classes = []
    for class_name in opt['classes']:
      classes.append(opt['classes'].index(class_name))

  for j in range(nframes):

      ret, img0 = video.read()
      if ret:
        img = letterbox(img0, imgsz, stride=stride)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
          img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment= False)[0]

        
        pred = non_max_suppression(pred, opt['conf-thres'], opt['iou-thres'], classes= classes, agnostic= False)
        t2 = time_synchronized()
        for i, det in enumerate(pred):
          s = ''
          s += '%gx%g ' % img.shape[2:]  # print string
          gn = torch.tensor(img0.shape)[[1, 0, 1, 0]]
          if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

            for c in det[:, -1].unique():
              n = (det[:, -1] == c).sum()  # detections per class
              s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
    
            for *xyxy, conf, cls in reversed(det):

              label = f'{names[int(cls)]} {conf:.2f}'
              plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)
        
        print(f"{j+1}/{nframes} frames processed")
        output.write(img0)
      else:
        break
    

output.release()
video.release()